In [2]:
# If cloning is needed -- i.e., if hangman.py and word counts are not already
# available in file, you can run this
!git clone https://github.com/ucsd-cse150a-w25/hw1.git
!cp hw1/hangman.py hangman.py
!cp hw1/hw1_word_counts_05.txt hw1_word_counts_05.txt

fatal: destination path 'hw1' already exists and is not an empty directory.


In [3]:
from hangman import hangman_game

# Play the game by yourself
hangman_game()



  +---+
  O   |
 /|\  |
 /    |
     ===

Word: T E S T S
Tried letters: A, E, I, O, S, T, U, Y
Congratulations! You guessed the word:  TESTS


1

In [41]:
import random

def random_inference(
    letters_tried: set[str],
    word_pattern: list[str],
    word_counts: dict[str, int]
) -> str:
    '''
    Random inference for playing hangman. This should be a simple method which returns a letter
    that is NOT in letters_tried but any other letter, at random.

    Hint: use the random.choice method
    '''
    # TODO: Implement random inference code to guess hangman
    # Check possible letters and randomize.
    letters = ['A','B','C','D','E','F','G','H','I','J','K','L','M',
               'N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    possible_letters = []
    # If the letter hasn't been used, append it to the list and randomly choose one
    for letter in letters:
        if letter not in letters_tried:
            possible_letters.append(letter)
    return random.choice(possible_letters)

hangman_game(random_inference)



  +---+
  O   |
 /|\  |
 / \  |
     ===

Word: G _ _ _ _
Tried letters: B, D, F, G, H, T, W
Game over! The word was:  GIVEN


0

In [56]:
from collections import defaultdict, Counter

def bayesian_inference(
    letters_tried: set[str],
    word_pattern: list[str],
    word_counts: dict[str, int]
) -> str:
    '''
    Improved Bayesian inference method for Hangman using posterior and predictive probabilities.
    '''
    # Step 1: Compute posterior probabilities P(W | O)
    probable_words = {}
    total_prior = 0

    for word, count in word_counts.items():
        # Ensure word matches the known pattern and doesn't contain incorrect guesses
        if all(wp == '_' or wp == w for wp, w in zip(word_pattern, word)) and \
           not any(letter in word and letter not in word_pattern for letter in letters_tried):
            probable_words[word] = count
            total_prior += count

    if total_prior == 0:
        # If no words match, pick a random letter that hasn’t been tried
        return random.choice(list(set('ABCDEFGHIJKLMNOPQRSTUVWXYZ') - letters_tried))

    # Normalize to get posterior probabilities
    posterior_probs = {word: count / total_prior for word, count in probable_words.items()}

    # Step 2: Compute predictive probabilities P(L | O)
    letter_probabilities = defaultdict(float)

    for word, prob in posterior_probs.items():
        for letter in set(word):  # Avoid duplicate counts within the same word
            if letter not in letters_tried:
                letter_probabilities[letter] += prob  # Weighted probability

    # Step 3: Choose the letter with the highest predictive probability
    return max(letter_probabilities, key=letter_probabilities.get, default=random.choice(list(set('ABCDEFGHIJKLMNOPQRSTUVWXYZ') - letters_tried)))


In [57]:
from typing import Optional, Callable
from tqdm import tqdm

def benchmark(
    inference: Optional[Callable],
    num_runs: int = 1000,
    seed: int = 0
) -> None:
    '''
    Benchmark method for testing out the bayesian inference method. The parameters given are:

    - inference: The function that should match the bayesian_inference() method above.
    - seed: The seed to pass into the hangman_game function.

    Return type: none.

    Print the accuracy out of num_runs. If the function throws an error, it should count as a fail.
    '''
    # TODO: Implement benchmark testing for a given function
    # Hint: use `random.seed(seed)` ONCE at the start
    # Hint: pass `interactive = False` into the hangman_game function to run faster without outputs.
    # Optional: use the `tqdm` module to keep track of the evaluation progress.

    random.seed(seed)
    correct = 0

    for _ in tqdm(range(num_runs)):
        try:
            result = hangman_game(inference, interactive = False)
            if result == 1:
                correct += 1
        except:
            pass


    accuracy = (correct / num_runs) * 100

    print(f"Accuracy: {accuracy}%")

benchmark(random_inference)
benchmark(bayesian_inference)

100%|██████████| 1000/1000 [00:03<00:00, 256.78it/s]


Accuracy: 0.4%


100%|██████████| 1000/1000 [01:12<00:00, 13.88it/s]

Accuracy: 2.8000000000000003%
